In [2]:
import os
import numpy as np
import cv2
import pandas as pd


from module.model import Model
from module.layers import Dense
from module.activations import ReLU, Softmax
from module.optimizers import Adam
from module.losses import CategoricalCrossentropy
from module.accuracy import Categorical_Accuracy


In [9]:
def one_hot(categories, data):
    cat_to_label = {cat: i for i, cat in enumerate(categories)}
    labels = np.array([cat_to_label[cat] for cat in data])
    return np.eye(len(categories))[labels]

In [28]:
data = pd.read_csv('data/digit-recognizer/train.csv') 

data = np.array(data)
m, n = data.shape
np.random.shuffle(data)
data_test = data[0:1000].T
y_test = data_test[0]
X_test = data_test[1:n]
X_test = X_test.T / 255.
data_train = data[1000:m].T
y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train.T / 255.
keys = np.array(range(X_train.shape[0]))
np.random.shuffle(keys)
X_train = X_train[keys]
y_train = y_train[keys]
category = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y_train = one_hot(category, y_train)
y_test = one_hot(category, y_test)

In [32]:
label_map = {cat: i for i, cat in enumerate(category)}

In [30]:
# Instantiate the model
model = Model()


# Add layers
model.add(Dense(X_train.shape[1], 128))
model.add(ReLU())
model.add(Dense(128, 128))
model.add(ReLU())
model.add(Dense(128, 10))
model.add(Softmax())

# Set loss, optimizer and accuracy objects
model.set(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(decay=1e-3),
    accuracy=Categorical_Accuracy()
)

# Finalize the model
model.finalize()

# Train the model
model.train(X_train, y_train, validation_data=(X_test, y_test),
            epochs=10, batch_size=128, print_every=100)

epoch: 1
step: 0, acc: 0.141, loss: 2.303 (data_loss: 2.303, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.758, loss: 0.624 (data_loss: 0.624, reg_loss: 0.000), lr: 0.0009090909090909091
step: 200, acc: 0.828, loss: 0.491 (data_loss: 0.491, reg_loss: 0.000), lr: 0.0008333333333333334
step: 300, acc: 0.914, loss: 0.245 (data_loss: 0.245, reg_loss: 0.000), lr: 0.0007692307692307692
step: 320, acc: 0.900, loss: 0.230 (data_loss: 0.230, reg_loss: 0.000), lr: 0.0007575757575757576
training, acc: 0.812, loss: 0.640 (data_loss: 0.640, reg_loss: 0.000), lr: 0.0007575757575757576
validation, acc: 0.901, loss: 0.309
epoch: 2
step: 0, acc: 0.922, loss: 0.274 (data_loss: 0.274, reg_loss: 0.000), lr: 0.000757002271006813
step: 100, acc: 0.906, loss: 0.290 (data_loss: 0.290, reg_loss: 0.000), lr: 0.0007037297677691766
step: 200, acc: 0.852, loss: 0.436 (data_loss: 0.436, reg_loss: 0.000), lr: 0.0006574621959237344
step: 300, acc: 0.930, loss: 0.174 (data_loss: 0.174, reg_loss: 0.000), lr: 0.00061690

In [37]:
# Predict on the image
confidences = model.predict(X_test)

# Get prediction instead of confidence levels
predictions = model.output_layer_activation.predictions(confidences)

# Get label name from label index
prediction = label_map[predictions[2]]

print(f'predicted: {prediction}')
print(f'true_value: {np.argmax(y_test[2])}')

predicted: 5
true_value: 5
